### Notes

entity -> dataframe \
attribute -> column \
primary key -> one (or more) columns in a table that are unique on every row \
prime attribute -> column that is part of the primary key \
atomic -> single data point within a cell (no lists or dictionaries in cells) \
\
"The key, the whole key, and nothing but the key. So help me Codd"

In [1]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

In [2]:
nba = pd.read_csv('nba.csv', low_memory=False)

## Original Data

In [3]:
pd.set_option('display.max_rows', 90)
nba.head(3).T

0  \
Unnamed: 0                                                                    0   
game_id                                                            202202170BRK   
game_date                                                            2022-02-17   
OT                                                                            0   
H_A                                                                           A   
Team_Abbrev                                                                 WAS   
Team_Score                                                                  117   
Team_pace                                                                  94.5   
Team_efg_pct                                                              0.627   
Team_tov_pct                                                               13.5   
Team_orb_pct                                                               22.9   
Team_ft_rate                                                              0.157   
Team_off_rtg                                                              123.8   
Inactives                          Joel Ayayi, Bradley Beal, Kristaps Porzingis   
Opponent_Abbrev                                                             BRK   
Opponent_Score                                                              103   
Opponent_pace                                                              94.5   
Opponent_efg_pct                                                          0.483   
Opponent_tov_pct                                                           13.1   
Opponent_orb_pct                                                           33.3   
Opponent_ft_rate                                                          0.191   
Opponent_off_rtg                                                          109.0   
player                                                            Corey Kispert   
player_id                                                             kispeco01   
starter                                                                       1   
mp                                                                        32:30   
fg                                                                            6   
fga                                                                           9   
fg_pct                                                                    0.667   
fg3                                                                           4   
fg3a                                                                          6   
fg3_pct                                                                   0.667   
ft                                                                            0   
fta                                                                           0   
ft_pct                                                                      0.0   
orb                                                                           0   
drb                                                                           1   
trb                                                                           1   
ast                                                                           1   
stl                                                                           2   
blk                                                                           0   
tov                                                                           2   
pf                                                                            2   
pts                                                                          16   
plus_minus                                                                   13   
did_not_play                                                                  0   
is_inactive                                                                   0   
ts_pct                                                                    0.889   
efg_pct    

## First Normal Form

In [4]:
#first normal form

nba = nba.drop(['Inactives'], axis = 1)
nba.head(3).T

0                  1  \
Unnamed: 0                                         0                  1   
game_id                                 202202170BRK       202202170BRK   
game_date                                 2022-02-17         2022-02-17   
OT                                                 0                  0   
H_A                                                A                  A   
Team_Abbrev                                      WAS                WAS   
Team_Score                                       117                117   
Team_pace                                       94.5               94.5   
Team_efg_pct                                   0.627              0.627   
Team_tov_pct                                    13.5               13.5   
Team_orb_pct                                    22.9               22.9   
Team_ft_rate                                   0.157              0.157   
Team_off_rtg                                   123.8              123.8   
Opponent_Abbrev                                  BRK                BRK   
Opponent_Score                                   103                103   
Opponent_pace                                   94.5               94.5   
Opponent_efg_pct                               0.483              0.483   
Opponent_tov_pct                                13.1               13.1   
Opponent_orb_pct                                33.3               33.3   
Opponent_ft_rate                               0.191              0.191   
Opponent_off_rtg                               109.0              109.0   
player                                 Corey Kispert         Kyle Kuzma   
player_id                                  kispeco01          kuzmaky01   
starter                                            1                  1   
mp                                             32:30              30:16   
fg                                                 6                  2   
fga                                                9                  7   
fg_pct                                         0.667              0.286   
fg3                                                4                  0   
fg3a                                               6                  3   
fg3_pct                                        0.667                0.0   
ft                                                 0                  1   
fta                                                0                  1   
ft_pct                                           0.0                1.0   
orb                                                0                  0   
drb                                                1                  7   
trb                                                1                  7   
ast                                                1                  5   
stl                                                2                  1   
blk                                                0                  0   
tov                                                2                  7   
pf                                                 2                  3   
pts                                               16                  5   
plus_minus                                        13                 10   
did_not_play                                       0                  0   
is_inactive                                        0                  0   
ts_pct                                         0.889              0.336   
efg_pct                                        0.889              0.286   
fg3a_per_fga_pct                               0.667              0.429   
fta_per_fga_pct                                  0.0              0.143   
orb_pct                                          0.0                0.0   
drb_pct                                          3.3               24.7   
trb_pct                                          1.8               13.9   
a

## Second Normal Form

In [5]:
#second normal form
## player name only depends on player id

players = nba[['player_id', 'player']].drop_duplicates()
players

player_id                    player
0       kispeco01             Corey Kispert
1       kuzmaky01                Kyle Kuzma
2       caldwke01  Kentavious Caldwell-Pope
3        netora01                 Raul Neto
4       bryanth01             Thomas Bryant
...           ...                       ...
109702  frazimi01           Michael Frazier
110441  howarwi01            William Howard
110913  mbahalu01          Luc Mbah a Moute
111399  bowmaky01                 Ky Bowman
112021  pargoje01              Jeremy Pargo

[812 rows x 2 columns]

In [6]:
nba = nba.drop(['player'], axis = 1)

In [7]:
## game date, overtime, season only depends on game id

games = nba[['game_id', 'game_date', 'OT', 'season']].drop_duplicates()
games

game_id   game_date  OT  season
0       202202170BRK  2022-02-17   0    2022
26      202202170CHO  2022-02-17   2    2022
48      202202170LAC  2022-02-17   0    2022
71      202202170MIL  2022-02-17   0    2022
95      202202170NOP  2022-02-17   0    2022
...              ...         ...  ..     ...
108259  202001080GSW  2020-01-08   0    2020
108887  202008020HOU  2020-08-02   0    2020
109683  201911060HOU  2019-11-06   0    2020
110125  201912250GSW  2019-12-25   0    2020
110643  202002200GSW  2020-02-20   0    2020

[3197 rows x 4 columns]

In [8]:
nba = nba.drop(['game_date', 'OT', "season"], axis=1)

In [9]:
nba.head(3).T

0                  1  \
Unnamed: 0                                         0                  1   
game_id                                 202202170BRK       202202170BRK   
H_A                                                A                  A   
Team_Abbrev                                      WAS                WAS   
Team_Score                                       117                117   
Team_pace                                       94.5               94.5   
Team_efg_pct                                   0.627              0.627   
Team_tov_pct                                    13.5               13.5   
Team_orb_pct                                    22.9               22.9   
Team_ft_rate                                   0.157              0.157   
Team_off_rtg                                   123.8              123.8   
Opponent_Abbrev                                  BRK                BRK   
Opponent_Score                                   103                103   
Opponent_pace                                   94.5               94.5   
Opponent_efg_pct                               0.483              0.483   
Opponent_tov_pct                                13.1               13.1   
Opponent_orb_pct                                33.3               33.3   
Opponent_ft_rate                               0.191              0.191   
Opponent_off_rtg                               109.0              109.0   
player_id                                  kispeco01          kuzmaky01   
starter                                            1                  1   
mp                                             32:30              30:16   
fg                                                 6                  2   
fga                                                9                  7   
fg_pct                                         0.667              0.286   
fg3                                                4                  0   
fg3a                                               6                  3   
fg3_pct                                        0.667                0.0   
ft                                                 0                  1   
fta                                                0                  1   
ft_pct                                           0.0                1.0   
orb                                                0                  0   
drb                                                1                  7   
trb                                                1                  7   
ast                                                1                  5   
stl                                                2                  1   
blk                                                0                  0   
tov                                                2                  7   
pf                                                 2                  3   
pts                                               16                  5   
plus_minus                                        13                 10   
did_not_play                                       0                  0   
is_inactive                                        0                  0   
ts_pct                                         0.889              0.336   
efg_pct                                        0.889              0.286   
fg3a_per_fga_pct                               0.667              0.429   
fta_per_fga_pct                                  0.0              0.143   
orb_pct                                          0.0                0.0   
drb_pct                                          3.3               24.7   
trb_pct                                          1.8               13.9   
ast_pct                                          4.0               18.5   
stl_pct                                          3.1                1.7   
blk_pct                                          0.0                0.0   
t

## Third Normal Form

In [10]:
nba.columns

Index(['Unnamed: 0', 'game_id', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev', 'Opponent_Score',
       'Opponent_pace', 'Opponent_efg_pct', 'Opponent_tov_pct',
       'Opponent_orb_pct', 'Opponent_ft_rate', 'Opponent_off_rtg', 'player_id',
       'starter', 'mp', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft',
       'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'pts', 'plus_minus', 'did_not_play', 'is_inactive', 'ts_pct', 'efg_pct',
       'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct',
       'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg',
       'def_rtg', 'bpm', 'minutes', 'double_double', 'triple_double', 'DKP',
       'FDP', 'SDP', 'DKP_per_minute', 'FDP_per_minute', 'SDP_per_minute',
       'pf_per_minute', 'ts', 'last_60_minutes_per_game_starting',
       'last_60_minutes_per_game_be

In [11]:
team_game = nba[['game_id','Team_Abbrev','H_A','Team_Score', 'Team_pace', 'Team_efg_pct', 'Team_tov_pct',
       'Team_orb_pct', 'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev']].drop_duplicates()

In [12]:
team_game.head(4).T

0             13            26            37
game_id          202202170BRK  202202170BRK  202202170CHO  202202170CHO
Team_Abbrev               WAS           BRK           MIA           CHO
H_A                         A             H             A             H
Team_Score                117           103           111           107
Team_pace                94.5          94.5          88.8          88.8
Team_efg_pct            0.627         0.483         0.471         0.453
Team_tov_pct             13.5          13.1          11.1          13.6
Team_orb_pct             22.9          33.3          26.8          28.1
Team_ft_rate            0.157         0.191         0.147         0.221
Team_off_rtg            123.8         109.0         103.4          99.7
Opponent_Abbrev           BRK           WAS           CHO           MIA

In [13]:
player_game = nba.drop(['Unnamed: 0','H_A', 'Team_Abbrev',
       'Team_Score', 'Team_pace', 'Team_efg_pct', 'Team_tov_pct',
       'Team_orb_pct', 'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev',
       'Opponent_Score', 'Opponent_pace', 'Opponent_efg_pct',
       'Opponent_tov_pct', 'Opponent_orb_pct', 'Opponent_ft_rate',
       'Opponent_off_rtg'], axis=1)

In [14]:
# now have player_game, players, game, team_game

In [15]:
players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 812 entries, 0 to 112021
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   player_id  812 non-null    object
 1   player     812 non-null    object
dtypes: object(2)
memory usage: 19.0+ KB


In [16]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3197 entries, 0 to 110643
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   game_id    3197 non-null   object
 1   game_date  3197 non-null   object
 2   OT         3197 non-null   int64 
 3   season     3197 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 124.9+ KB


In [17]:
dbserver = psycopg2.connect(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = '5432'
)

dbserver.autocommit = True

In [18]:
cursor = dbserver.cursor()
#conduit through which you can now speak to postgres

In [19]:
#do this in case the database already exists, want to drop it then recreate from scratch
try:
    cursor.execute('CREATE DATABASE nba')
except:
    cursor.execute('DROP DATABASE nba')
    cursor.execute('CREATE DATABASE nba')

In [20]:
engine = create_engine("postgresql+psycopg2://{user}:{pw}@{service}/{db}".format(
    user = 'postgres',
    pw = POSTGRES_PASSWORD,
    service = 'postgres',
    db = 'nba'
))

In [30]:
#make all cols lowercase here

In [21]:
games.to_sql('games', con=engine, index=False, chunksize=1000, if_exists='replace')

3197

In [22]:
players.to_sql('players', con=engine, index=False, chunksize=1000, if_exists='replace')

812

In [23]:
team_game.to_sql('team_game', con=engine, index=False, chunksize=1000, if_exists='replace')

6394

In [ ]:
player_game.to_sql('player_game', con=engine, index=False, chunksize=500, if_exists='replace')

In [25]:
myquery = '''
SELECT *
FROM games
'''
pd.read_sql_query(myquery, con=engine)

game_id   game_date  OT  season
0     202202170BRK  2022-02-17   0    2022
1     202202170CHO  2022-02-17   2    2022
2     202202170LAC  2022-02-17   0    2022
3     202202170MIL  2022-02-17   0    2022
4     202202170NOP  2022-02-17   0    2022
...            ...         ...  ..     ...
3192  202001080GSW  2020-01-08   0    2020
3193  202008020HOU  2020-08-02   0    2020
3194  201911060HOU  2019-11-06   0    2020
3195  201912250GSW  2019-12-25   0    2020
3196  202002200GSW  2020-02-20   0    2020

[3197 rows x 4 columns]

In [26]:
myquery = '''
SELECT *
FROM games
WHERE season = 2022
'''
pd.read_sql_query(myquery, con=engine)

game_id   game_date  OT  season
0    202202170BRK  2022-02-17   0    2022
1    202202170CHO  2022-02-17   2    2022
2    202202170LAC  2022-02-17   0    2022
3    202202170MIL  2022-02-17   0    2022
4    202202170NOP  2022-02-17   0    2022
..            ...         ...  ..     ...
878  202111190NOP  2021-11-19   0    2022
879  202111290LAC  2021-11-29   0    2022
880  202201130NOP  2022-01-13   0    2022
881  202201250PHI  2022-01-25   0    2022
882  202201210PHI  2022-01-21   0    2022

[883 rows x 4 columns]

In [28]:
# all games where a player scores > 40 points, and team > 120 points, and calculate % of team scores

In [29]:
#won't work unless make all col names lowercase, but don't need to worry about this

myquery = '''
SELECT pg.game_id, pg.player_id, pg.pts
FROM player_game pg
INNER JOIN team_game tg
    ON pg.game_id = tg.game_id
WHERE pts > 40
'''
pd.read_sql_query(myquery, con=engine)

game_id  player_id  starter        mp  fg  fga  fg_pct  fg3  fg3a  \
0    202202170MIL  embiijo01        1     37:03  14   21   0.667    3     4   
1    202202170NOP  doncilu01        1     38:02  17   35   0.486    7    14   
2    202110250CHO  tatumja01        1     41:28  14   28   0.500    6    12   
3    202112130BOS  tatumja01        1     38:04  16   25   0.640    7    13   
4    202201230WAS  tatumja01        1     32:52  18   28   0.643    9    14   
..            ...        ...      ...       ...  ..  ...     ...  ...   ...   
432  202008090SAC  riverau01        0  33:26:00  14   20   0.700    6    11   
433  202008090SAC  riverau01        0  33:26:00  14   20   0.700    6    11   
434  202008120HOU  hardeja01        1  35:28:00  13   21   0.619    7    14   
435  202008120HOU  hardeja01        1  35:28:00  13   21   0.619    7    14   
436  201911080MIN  russeda01        1  40:05:00  19   37   0.514    7    17   

     fg3_pct  ...  pf_per_minute     ts  last_60_minutes_per_game_starting  \
0      0.750  ...       0.134953  27.16                          32.651282   
1      0.500  ...       0.078878  40.28                          35.731061   
2      0.500  ...       0.072347  31.08                          37.283333   
3      0.538  ...       0.026270  27.20                          36.251852   
4      0.643  ...       0.030426  30.64                          36.879333   
..       ...  ...            ...    ...                                ...   
432    0.545  ...       0.089731  23.52                          36.770382   
433    0.545  ...       0.089731  23.52                          36.770382   
434    0.500  ...       0.056391  27.16                          35.923333   
435    0.500  ...       0.056391  27.16                          35.923333   
436    0.412  ...       0.000000  40.52                          30.423333   

     last_60_minutes_per_game_bench   PG%   SG%   SF%   PF%     C%  \
0                         17.575562   0.0   0.0   0.0   0.0  100.0   
1                         19.233348  35.0  62.0   3.0   0.0    0.0   
2                         20.157981   0.0   0.0  61.0  39.0    0.0   
3                         19.600289   0.0   0.0  61.0  39.0    0.0   
4                         19.939550   0.0   0.0  61.0  39.0    0.0   
..                              ...   ...   ...   ...   ...    ...   
432                       21.358333  49.0  49.0   2.0   0.0    0.0   
433                       21.358333  36.0  53.0  11.0   0.0    0.0   
434                       20.866319  16.0  75.0  10.0   0.0    0.0   
435                       20.866319  31.0  54.0  13.0   2.0    0.0   
436                        0.000000  87.0  13.0   0.0   0.0    0.0   

     active_position_minutes  
0                  48.328497  
1                  61.379802  
2                  53.599860  
3                  53.392579  
4                  56.027039  
..                       ...  
432                      NaN  
433                      NaN  
434                      NaN  
435                      NaN  
436                61.251536  

[437 rows x 59 columns]